In [1]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection._split import train_test_split
# from sklearn.linear_model.stochastic_gradient import SGDClassifier
from hazm import word_tokenize
from hazm.Stemmer import Stemmer
from keras import models, layers
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.feature_selection.univariate_selection import SelectPercentile
# from sklearn.feature_selection import chi2
import numpy
# from sklearn import svm
# from sklearn.neighbors.classification import KNeighborsClassifier
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.ensemble.weight_boosting import AdaBoostClassifier
# from sklearn.ensemble.forest import RandomForestClassifier
# from sklearn.ensemble import VotingClassifier
from sklearn.metrics.classification import f1_score, precision_score, recall_score
# from sklearn.neural_network import MLPClassifier, BernoulliRBM
from sklearn.metrics import zero_one_loss
from sklearn.metrics import confusion_matrix

nltk.download('stopwords')
nltk.download('punkt')

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Izad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Izad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
perisca_dataset = pd.read_csv("per.csv", encoding="UTF-8", header=0)
perisca_dataset.head()

,NewsID,Title,Body,Date,Time,Category,Category2
0,843656,\nوزير علوم درجمع استادان نمونه: سن بازنشستگي ...,\nوزير علوم در جمع استادان نمونه كشور گفت: از ...,\n138/5//09,\n0:9::18,\nآموزشي-,\nآموزشي
1,837144,\nگردهمايي دانش‌آموختگان موسسه آموزش عالي سوره...,\nبه گزارش سرويس صنفي آموزشي خبرگزاري دانشجويا...,\n138/5//09,\n1:4::11,\nآموزشي-,\nآموزشي
2,436862,\nنتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور...,\nنتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و...,\n138/3//07,\n1:0::03,\nآموزشي-,\nآموزشي
3,227781,\nهمايش يكروزه آسيب شناسي مفهوم روابط عمومي در...,\n,\n138/2//02,\n1:3::42,\nاجتماعي-خانواده-,\nاجتماعي
4,174187,\nوضعيت اقتصادي و ميزان تحصيلات والدين از مهمت...,\nمحمدتقي علوي يزدي، مجري اين طرح پژوهشي در اي...,\n138/1//08,\n1:1::49,\nآموزشي-,\nآموزشي


In [3]:
#stopwords from: https://raw.githubusercontent.com/kharazi/persian-stopwords/master/persian
# NLTK : Natural Language Toolkit
with open('stopwords.txt', encoding="UTF-8") as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [str(line).replace('\n', '') for line in stopwords]

# insert NLTK English stopwords into nltk_stopwords
nltk_stopwords = nltk.corpus.stopwords.words('english')
# appent presian stopwords in nltk_stopwords
nltk_stopwords.extend(stopwords)

In [4]:
len(nltk_stopwords)

1495

In [5]:
stemmer = Stemmer()
dataset = pd.DataFrame(columns=('title_body', 'category'))

In [6]:
# We can make a dataframe with the concatination of series
for index, row in perisca_dataset.iterrows():
    title_body = row['Title'] + ' ' + row['Body']
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in nltk_stopwords]
    title_body_tokenized_filtered_stemming = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    dataset.loc[index] = {'title_body': ' '.join(title_body_tokenized_filtered_stemming), 'category': row['Category2']}

dataset.head()

,title_body,category
0,وزير علو درجمع استاد نمونه سن بازنشستگي استاد ...,\nآموزشي
1,گردهمايي دانش‌آموختگ موسسه آموز عالي سوره برگز...,\nآموزشي
2,نتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور ن...,\nآموزشي
3,هماي يكروزه آسيب شناسي مفهو روابط عمومي بابلسر...,\nاجتماعي
4,وضعي اقتصادي ميز تحصيل والدين مهمترين عوامل مو...,\nآموزشي


In [7]:
# df = Document Frequency
# idf=>  inverted doument frequency => 1/df
# Tfidf =>  Tf * idf
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5)
X = vectorizer.fit(dataset['title_body']).transform(dataset['title_body']) # Bag of Words
#--------------------------------------------
# Why fit is separate from transform? 
# Fit only applys on training data, but transform would apply on both train and test data

In [8]:
le = preprocessing.LabelEncoder()
y = le.fit(dataset['category']).transform(dataset['category'])
numpy.unique(dataset['category'])


array(['\nآموزشي', '\nاجتماعي', '\nاقتصادي', '\nبهداشتي', '\nتاريخي',
       '\nسياسي', '\nعلمي', '\nفرهنگي', '\nفقه و حقوق', '\nمذهبي',
       '\nورزشي'], dtype=object)

In [22]:
numpy.shape(X)

(10999, 60798)

In [23]:
numpy.shape(y)

(10999,)

In [9]:
KX_train, KX_test, Ky_train, Ky_test = train_test_split(X, to_categorical(y, 11))
model = models.Sequential()
model.add(layers.Dense(200, activation="relu", input_shape=(60798,)))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(200, activation="relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(200, activation="relu"))
model.add(layers.Dense(11, activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 200)               12159800  
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_4 (Dense)              (None, 11)                2211      
Total params: 12,242,411
Trainable params: 12,242,411
Non-trainable params: 0
__________________________________________

In [10]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
results = model.fit(KX_train, Ky_train, epochs=5, batch_size=500, validation_data=(KX_test, Ky_test))
print("Deep acc: ", numpy.mean(results.history["val_accuracy"]))

Train on 8249 samples, validate on 2750 samples
Epoch 1/5
8249/8249 [==============================] - 17s 2ms/step - loss: 0.5283 - accuracy: 0.8932 - val_loss: 0.3513 - val_accuracy: 0.9091
Epoch 2/5
8249/8249 [==============================] - 16s 2ms/step - loss: 0.3362 - accuracy: 0.9091 - val_loss: 0.3196 - val_accuracy: 0.9091
Epoch 3/5
8249/8249 [==============================] - 16s 2ms/step - loss: 0.3026 - accuracy: 0.9091 - val_loss: 0.2849 - val_accuracy: 0.9091
Epoch 4/5
8249/8249 [==============================] - 16s 2ms/step - loss: 0.2598 - accuracy: 0.9107 - val_loss: 0.2237 - val_accuracy: 0.9187
Epoch 5/5
8249/8249 [==============================] - 16s 2ms/step - loss: 0.1817 - accuracy: 0.9291 - val_loss: 0.1480 - val_accuracy: 0.9448
Deep acc:  0.9181554913520813


In [44]:
KX_train, KX_test, Ky_train, Ky_test = train_test_split(X, to_categorical(y, 11))
model = models.Sequential()
model.add(layers.Dense(500, activation="relu", input_shape=(60798,)))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(150, activation="relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(100, activation="relu"))
model.add(layers.Dense(11, activation="sigmoid"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 500)               30399500  
_________________________________________________________________
dropout_3 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 150)               75150     
_________________________________________________________________
dropout_4 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               15100     
_________________________________________________________________
dense_8 (Dense)              (None, 11)                1111      
Total params: 30,490,861
Trainable params: 30,490,861
Non-trainable params: 0
__________________________________________

In [45]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
results = model.fit(KX_train, Ky_train, epochs=5, batch_size=500, validation_data=(KX_test, Ky_test))
print("Deep acc: ", numpy.mean(results.history["val_accuracy"]))

Train on 8249 samples, validate on 2750 samples
Epoch 1/5
8249/8249 [==============================] - 32s 4ms/step - loss: 0.5241 - accuracy: 0.8803 - val_loss: 0.3458 - val_accuracy: 0.9091
Epoch 2/5
8249/8249 [==============================] - 33s 4ms/step - loss: 0.3349 - accuracy: 0.9091 - val_loss: 0.3139 - val_accuracy: 0.9091
Epoch 3/5
8249/8249 [==============================] - 31s 4ms/step - loss: 0.2916 - accuracy: 0.9093 - val_loss: 0.2641 - val_accuracy: 0.9115
Epoch 4/5
8249/8249 [==============================] - 27s 3ms/step - loss: 0.2401 - accuracy: 0.9138 - val_loss: 0.2091 - val_accuracy: 0.9180
Epoch 5/5
8249/8249 [==============================] - 29s 3ms/step - loss: 0.1741 - accuracy: 0.9342 - val_loss: 0.1489 - val_accuracy: 0.9446
Deep acc:  0.918439757823944
